In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import Dash, html, dcc
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

df = pd.read_csv(r"D:\e23\StudentsPerformance.csv")
df = df.drop(columns=['race/ethnicity'])
matiere = ["math score", "reading score", "writing score"]
df['score_moyen'] = df[matiere].mean(axis=1).round(2)

years = sorted(df['year'].unique())
states = sorted(df['state'].unique())
genres = df['gender'].unique()

app = Dash(__name__)
server = app.server

app.layout = html.Div([
    # Titre principal
    html.H1(
        "Tableau de bord Performances Scolaires (USA)",
        style={'textAlign': 'center', 'padding': '20px 0', 'color': '#003366', 'font-family': 'Segoe UI'}
    ),

    # KPIs
    html.Div([
        html.Div([
            html.H3("Score moyen général", style={'font-family': 'Segoe UI'}),
            html.P(f"{df['score_moyen'].mean():.2f}", style={'fontSize': '30px', 'color': '#1f77b4', 'font-family': 'Segoe UI'})
        ], className='kpi', style={'padding': '10px', 'width': '30%', 'display': 'inline-block'}),
        html.Div([
            html.H3("Score moyen max", style={'font-family': 'Segoe UI'}),
            html.P(f"{df['score_moyen'].max():.2f}", style={'fontSize': '30px', 'color': '#2ca02c', 'font-family': 'Segoe UI'})
        ], className='kpi', style={'padding': '10px', 'width': '30%', 'display': 'inline-block'}),
        html.Div([
            html.H3("Score moyen min", style={'font-family': 'Segoe UI'}),
            html.P(f"{df['score_moyen'].min():.2f}", style={'fontSize': '30px', 'color': '#d62728', 'font-family': 'Segoe UI'})
        ], className='kpi', style={'padding': '10px', 'width': '30%', 'display': 'inline-block'}),
    ], style={'textAlign': 'center', 'marginBottom': '30px'}),

    # Onglets
    dcc.Tabs([
        dcc.Tab(label="Score moyen par État", children=[
            html.Br(),
            html.Label("Sélectionner l'année", style={'font-family': 'Segoe UI'}),
            dcc.Dropdown(
                id='year_dropdown',
                value=years[0],
                options=[{'label': y, 'value': y} for y in years]
            ),
            dcc.Graph(id='state_scores_chart')
        ]),

        dcc.Tab(label="Évolution par État", children=[
            html.Br(),
            html.Label("Sélectionner un ou plusieurs États", style={'font-family': 'Segoe UI'}),
            dcc.Dropdown(
                id='state_dropdown',
                multi=True,
                options=[{'label': s, 'value': s} for s in states]
            ),
            dcc.Graph(id='state_trend_chart')
        ]),

        dcc.Tab(label="Analyse globale", children=[
            html.Br(),
            dcc.Graph(
                figure=px.scatter(
                    df,
                    x='reading score',
                    y='math score',
                    color='gender',
                    size='score_moyen',
                    hover_name='state',
                    title='Lecture vs Math (taille = score moyen)',
                    labels={'reading score':'Reading', 'math score':'Math'},
                )
            ),
            html.Br(),
            # Histogrammes des scores par matière
            dcc.Graph(
                figure=px.histogram(
                    df,
                    x=matiere,
                    barmode='overlay',
                    title='Distribution des scores par matière',
                    labels={col: col for col in matiere}
                )
            ),
            html.Br(),
            # Boxplot par genre
            dcc.Graph(
                figure=px.box(
                    df,
                    x='gender',
                    y='score_moyen',
                    color='gender',
                    title='Répartition du score moyen par genre'
                )
            ),
            html.Br(),
            # Heatmap de corrélation
            dcc.Graph(
                figure=px.imshow(
                    df[matiere + ['score_moyen']].corr(),
                    text_auto=True,
                    title='Corrélation entre les matières'
                )
            )
        ])
    ])
], style={'width': '90%', 'margin': 'auto', 'font-family': 'Segoe UI'})


@app.callback(
    Output('state_scores_chart', 'figure'),
    Input('year_dropdown', 'value')
)
def update_state_scores(selected_year):
    year_df = df[df['year'] == selected_year]
    state_avg = year_df.groupby('state')['score_moyen'].mean().reset_index()

    fig = px.bar(
        state_avg,
        x='score_moyen',
        y='state',
        orientation='h',
        text='score_moyen',
        title=f"Score moyen par État — {selected_year}",
        height=500
    )
    fig.update_layout(yaxis={'categoryorder':'total ascending'})
    return fig

@app.callback(
    Output('state_trend_chart', 'figure'),
    Input('state_dropdown', 'value')
)
def update_trend(states_selected):
    if not states_selected:
        raise PreventUpdate

    subset = df[df['state'].isin(states_selected)]
    state_year_avg = subset.groupby(['state','year'])['score_moyen'].mean().reset_index()

    fig = px.line(
        state_year_avg,
        x='year',
        y='score_moyen',
        color='state',
        markers=True,
        title=f"Évolution du score moyen — {', '.join(states_selected)}"
    )
    return fig

# -------------------------------------------------------------
# 5. Run server
# -------------------------------------------------------------
if __name__ == '__main__':
    app.run(jupyter_mode = "tab",debug=True, port=8050)


ModuleNotFoundError: No module named 'dash'